In [ ]:
from ultralytics import YOLO

In [ ]:
MODEL_PATH = 'best.pt'
model = YOLO(MODEL_PATH)


In [ ]:
model.export(format="torchscript")
model.export(format="onnx")  # export the model to ONNX format

In [ ]:
model.export(format="torchscript")
model.export(format="onnx")  # export the model to ONNX format

In [ ]:
import onnxruntime
import numpy as np
import torch

ONNX_MODEL_PATH = '/home/andrew/Andrew/yolov9/UltraSegmTrainFISHIAL_OBJECT_DETECTION/first_test_9e5/weights/best.onnx'
TS_MODEL_PATH   = '/home/andrew/Andrew/yolov9/UltraSegmTrainFISHIAL_OBJECT_DETECTION/first_test_9e5/weights/best.torchscript'

torch_model = torch.jit.load(TS_MODEL_PATH)

ort_session = onnxruntime.InferenceSession(ONNX_MODEL_PATH)

In [ ]:
batch_size = 1
x = torch.randn(batch_size, 3, 640, 640, requires_grad=False)

# compute ONNX Runtime output prediction
ort_inputs = {
    ort_session.get_inputs()[0].name: x.numpy()
}
ort_outs = ort_session.run(None, ort_inputs)[0]

torch_out = model.model(torch.randn(1, 3, 640, 640))[0]

full_model_output = torch_out.numpy()

print(np.sum(ort_outs - full_model_output))

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(ort_outs, full_model_output, rtol=1e-03, atol=1e-05)

In [ ]:
ort_session.get_inputs()[0].name

In [ ]:
torch_out[0].shape

In [ ]:
import fiftyone as fo
import cv2
import matplotlib.pyplot as plt

data = fo.load_dataset('SEGM-2024-V0.8-VALIDATION')
# data = fo.load_dataset('SEGM-2024-V0.8')

count_to_visualize = 20


In [ ]:
view = data.take(count_to_visualize)

for sample in data:
    img_path = sample.filepath
    
    result = model.predict(img_path, save=False, imgsz=640, conf=0.5, iou=0.5)
    np_img = cv2.imread(img_path)
    for boxes in result:
        for box in boxes.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cv2.rectangle(np_img,(x1, y1),(x2,y2),(0, 0, 255), 2)
            
    plt.imshow(np_img)
    plt.show()
    
#     break
    

In [ ]:
img_path = 'for_test.jpg'
result = model.predict(img_path, save=False, imgsz=640, conf=0.1, iou=0.4)
np_img = cv2.imread(img_path)
for boxes in result:
    for box in boxes.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        cv2.rectangle(np_img,(x1, y1),(x2,y2),(0, 0, 255), 2)

plt.imshow(np_img)
plt.show()